In [1]:
import cassandra

In [2]:
from cassandra.cluster import Cluster
clstr= Cluster()
session= clstr.connect()

# create keyspace

In [6]:
session.execute("create keyspace csd_db2 with replication={'class': 'SimpleStrategy', 'replication_factor' : 3}")

# create table

In [10]:
session.execute("create table csd_db2.student(studentID int,name text,age int, primary key(studentID))")

# insert values

In [18]:
session.execute("insert into csd_db2.student(studentID, name, age) values(6,'priya6',59)")

In [19]:
row= session.execute("select * from csd_db2.student")
for i in row:
    print(i)

Row(studentid=5, age=99, name='priya5')
Row(studentid=1, age=26, name='priya')
Row(studentid=2, age=2, name='priya1')
Row(studentid=4, age=9, name='priya4')
Row(studentid=6, age=59, name='priya6')
Row(studentid=3, age=29, name='priya3')


# allow filtering

In [21]:
row= session.execute("select * from csd_db2.student where age>20 allow filtering")
for i in row:
    print(i)

Row(studentid=5, age=99, name='priya5')
Row(studentid=1, age=26, name='priya')
Row(studentid=6, age=59, name='priya6')
Row(studentid=3, age=29, name='priya3')


# BatchStatement

In [22]:
from cassandra.query import SimpleStatement, BatchStatement

In [27]:
batch=BatchStatement()
studentlist=[(7,'priya7',34),(8,'priya8',23)]
for i in studentlist:
    batch.add(SimpleStatement('insert into csd_db2.student(studentID,name,age) values(%s,%s,%s)'),(i[0],i[1],i[2]))
              
session.execute(batch)

In [28]:
row= session.execute("select * from csd_db2.student")
for i in row:
    print(i)

Row(studentid=5, age=99, name='priya5')
Row(studentid=1, age=26, name='priya')
Row(studentid=8, age=23, name='priya8')
Row(studentid=2, age=2, name='priya1')
Row(studentid=4, age=9, name='priya4')
Row(studentid=7, age=34, name='priya7')
Row(studentid=6, age=59, name='priya6')
Row(studentid=3, age=29, name='priya3')


# prepare statement

In [30]:
smt=session.prepare('insert into csd_db2.student (studentID,name,age) values(?,?,?)')
session.execute(smt.bind([9,'priya9',28]))

In [33]:
row=session.execute("select * from csd_db2.student")
for i in row:
    print(i)

Row(studentid=5, age=99, name='priya5')
Row(studentid=1, age=26, name='priya')
Row(studentid=8, age=23, name='priya8')
Row(studentid=2, age=2, name='priya1')
Row(studentid=4, age=9, name='priya4')
Row(studentid=7, age=34, name='priya7')
Row(studentid=6, age=59, name='priya6')
Row(studentid=9, age=28, name='priya9')
Row(studentid=3, age=29, name='priya3')


In [35]:
row=session.execute("select * from csd_db2.student where studentID=5")
for i in row:
    print(i)

Row(studentid=5, age=99, name='priya5')


# update

In [36]:
session.execute("update csd_db2.student set name='abhi' where studentID=2")

In [59]:
row=session.execute("select * from csd_db2.student where age=55 allow filtering")
for i in row:
    print(i)

Row(studentid=2, age=55, name='abhi')


In [38]:
session.execute("update csd_db2.student set name='abhi', age=55 where studentID=2")

In [39]:
row=session.execute("select * from csd_db2.student where studentID=2")
for i in row:
    print(i)

Row(studentid=2, age=55, name='abhi')


# delete

In [40]:
session.execute("delete from csd_db2.student where studentID=9")

In [41]:
row=session.execute("select * from csd_db2.student")
for i in row:
    print(i)

Row(studentid=5, age=99, name='priya5')
Row(studentid=1, age=26, name='priya')
Row(studentid=8, age=23, name='priya8')
Row(studentid=2, age=55, name='abhi')
Row(studentid=4, age=9, name='priya4')
Row(studentid=7, age=34, name='priya7')
Row(studentid=6, age=59, name='priya6')
Row(studentid=3, age=29, name='priya3')


# orderby
You can fine-tune the display order using the ORDER BY clause. The partition key must be defined in the WHERE clause and the ORDER BY clause defines the clustering column to use for ordering.

both partition and clustering keys has to be given

In [49]:
session.execute("create table csd_db2.student1(studentID int,name text,age int, primary key(studentID,age))")

In [50]:
batch=BatchStatement()
studentlist=[(7,'priya7',34),(8,'priya8',23),(1,'priya8',23),(8,'priya8',24)]
for i in studentlist:
    batch.add(SimpleStatement('insert into csd_db2.student1(studentID,name,age) values(%s,%s,%s)'),(i[0],i[1],i[2]))
              
session.execute(batch)

In [51]:
rows=session.execute('select * from csd_db2.student1')
for i in rows:
    print(i)

Row(studentid=1, age=23, name='priya8')
Row(studentid=8, age=23, name='priya8')
Row(studentid=8, age=24, name='priya8')
Row(studentid=7, age=34, name='priya7')


In [61]:
rows=session.execute('select * from csd_db2.student1 where studentID = 8 order by age DESC')

In [62]:
for i in rows:
    print (i)

Row(studentid=8, age=24, name='priya8')
Row(studentid=8, age=23, name='priya8')


In [64]:
row= session.execute("select * from csd_db2.student1 where studentID=8 and age>23 ")
for i in row:
    print(i)

Row(studentid=8, age=24, name='priya8')
